<a href="https://colab.research.google.com/github/JoaoMMS98/ML_2.0/blob/main/Optuna_hyperparams_before_FS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install optuna==3.0.0 \
            #  scikit-learn==1.3.0 \
            #  xgboost==1.7.6 \
            #  numpy==1.25.2 \
            #  torch==2.0.1

In [2]:
import pandas as pd

train_full = pd.read_csv('traindf (2).csv')
val_full = pd.read_csv('valdf (2).csv')
test_scaled = pd.read_csv('testdf (2).csv')

In [3]:
# # Divide datasets by 30 for speedtest
# train_full = train_full.iloc[::50, :]
# val_full = val_full.iloc[::50, :]
# test_scaled = test_scaled.iloc[::50, :]

In [4]:
merged_df = pd.concat([train_full, val_full], ignore_index=True)

In [5]:
merged_df

,Claim Injury Type,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,4. TEMPORARY,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.274579,1.046863,...,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,3. MED ONLY,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,0.384500,1.046863,...,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,2. NON-COMP,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.040655,1.046863,...,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,2. NON-COMP,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.040655,1.046863,...,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,3. MED ONLY,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,0.574898,1.046863,...,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574020,4. TEMPORARY,0.173915,1.273293,0.229535,0.142837,0.558209,-0.518970,-0.780636,-1.416376,-1.261856,...,0,0,1,-1.601871,0.073562,-0.913279,0,0,0,0
574021,2. NON-COMP,0.085024,-0.755254,0.374636,0.060851,-1.026043,2.021605,-0.895805,1.274579,1.046863,...,0,0,0,-0.223769,1.753521,3.045807,0,0,0,0
574022,3. MED ONLY,-0.876899,-0.755254,-0.133220,-0.761740,0.558209,-0.622081,-0.780636,-0.959634,-0.667644,...,0,0,1,0.584317,1.572367,-0.040737,0,0,0,0
574023,4. TEMPORARY,1.666008,1.303004,-1.511687,1.421816,0.580704,-0.577546,0.983139,0.574898,1.046863,...,0,0,1,0.125125,0.021484,0.774823,0,0,0,0


In [6]:
# Create a copy of merged_df
cleaned_df = merged_df.copy()

# Drop the specified columns
cleaned_df = cleaned_df.drop(columns=[
    'Zip Code',
    'Alternative Dispute Resolution_U',
    'Claim Injury Type', # drop target
    'Agreement Reached',
    'County of Injury'
], errors='ignore')

# Keep only numeric columns
analysis_df = cleaned_df.select_dtypes(include=['number'])

# Similarly, drop columns from test_scaled
test_scaled = test_scaled.drop(columns=[
    'Zip Code',
    'Alternative Dispute Resolution_U',
    'Agreement Reached',
    'County of Injury'
], errors='ignore')

#Keep only numerical
test_scaled = test_scaled.select_dtypes(include=['number'])

In [7]:
analysis_df

,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,District Name,First Hearing Date,IME-4 Count,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.046863,-0.037533,0.920180,...,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,1.046863,-1.477658,-0.445038,...,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.046863,-0.037533,-0.445038,...,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.046863,-0.037533,-0.445038,...,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,1.046863,-1.413414,-0.445038,...,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574020,0.173915,1.273293,0.229535,0.142837,0.558209,-0.518970,-0.780636,-1.261856,-0.037533,-0.445038,...,0,0,1,-1.601871,0.073562,-0.913279,0,0,0,0
574021,0.085024,-0.755254,0.374636,0.060851,-1.026043,2.021605,-0.895805,1.046863,-0.037533,-0.445038,...,0,0,0,-0.223769,1.753521,3.045807,0,0,0,0
574022,-0.876899,-0.755254,-0.133220,-0.761740,0.558209,-0.622081,-0.780636,-0.667644,-0.037533,-0.445038,...,0,0,1,0.584317,1.572367,-0.040737,0,0,0,0
574023,1.666008,1.303004,-1.511687,1.421816,0.580704,-0.577546,0.983139,1.046863,2.789179,3.650616,...,0,0,1,0.125125,0.021484,0.774823,0,0,0,0


In [8]:
test_scaled

,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,District Name,First Hearing Date,IME-4 Count,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,-1.686439,-0.755254,1.825655,-1.458620,0.558209,-0.411805,0.983139,1.046863,-0.037533,-0.445038,...,0,0,1,0.343719,-1.434468,1.148770,0,0,0,0
1,-1.686439,-0.755254,1.825655,-1.458620,0.558209,-0.362470,0.983139,1.046863,-0.037533,-0.445038,...,0,0,0,-0.240232,-1.024995,0.340333,0,0,0,0
2,-1.686439,-0.755254,-1.076382,-1.453154,0.558209,-0.623482,0.983139,1.046863,-0.037533,-0.445038,...,0,0,0,-1.204931,-0.950851,-0.671104,0,0,0,0
3,-1.686439,-0.755254,-143.493821,-1.458620,0.558209,-0.283982,0.983139,1.046863,-0.037533,-0.445038,...,0,0,0,0.521399,1.395544,-0.240175,0,0,0,0
4,-1.686439,-0.755254,1.390349,-1.453154,0.558209,-0.322674,0.983139,1.046863,-0.037533,-0.445038,...,0,0,1,-0.952494,-0.405208,1.347021,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387970,-3.337265,-0.755254,-1.294035,8.715824,0.558209,2.021605,-0.895805,-1.049064,-0.037533,-0.445038,...,0,0,1,-1.032355,-0.252428,-0.706718,1,0,1,1
387971,-3.337265,-0.755254,-0.931280,0.183830,-4.483761,-0.430883,-0.780636,-0.901458,-0.037533,-0.445038,...,0,0,0,-1.032355,-0.252428,0.091035,1,0,1,1
387972,-3.337265,-0.755254,0.084433,0.183830,0.558209,2.021605,-0.895805,1.046863,-0.037533,-0.445038,...,0,0,1,-1.032355,-0.252428,1.399254,1,0,1,1
387973,-3.337265,-0.755254,0.229535,0.183830,-4.461267,-0.453747,-1.524399,1.046863,-0.037533,-0.445038,...,0,0,1,-1.032355,-0.252428,0.162263,1,0,1,1


In [9]:
y = merged_df['Claim Injury Type']
claim_injury_type = merged_df['Claim Injury Type']

In [10]:
# import optuna
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import f1_score
# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from sklearn.preprocessing import LabelEncoder
# import numpy as np
# from optuna import TrialPruned
# import gc  # CRITICAL: QUANTUM MEMORY MANAGEMENT

# # QUANTUM MEMORY OPTIMIZATION
# def quantum_memory_cleanup():
#     gc.collect()
#     import torch
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()

# # Data preprocessing remains quantum-stable
# label_encoder = LabelEncoder()
# y = label_encoder.fit_transform(y)

# X_train, X_val, y_train, y_val = train_test_split(
#     analysis_df, y,
#     test_size=0.2,
#     random_state=42,
#     stratify=y
# )

# def objective(trial):
#     # CRITICAL GPU OPTIMIZATION PARAMETERS
#     params = {
#         # QUANTUM-OPTIMIZED PARAMETER RANGES
#         'max_depth': trial.suggest_int('max_depth', 6, 12),  # CRITICAL ADJUSTMENT!
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
#         'min_child_weight': trial.suggest_float('min_child_weight', 1, 20),
#         'subsample': trial.suggest_float('subsample', 0.6, 0.8),
#         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.8),
#         'n_estimators': trial.suggest_int('n_estimators', 100, 400),  # EFFICIENCY QUANTUM LEAP
#         'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
#         'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),

#         # CRITICAL GPU ACCELERATORS
#         'tree_method': 'gpu_hist',  # QUANTUM GPU ACCELERATION
#         'predictor': 'gpu_predictor',
#         'objective': 'multi:softprob',
#         'random_state': 42
#     }

#     cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
#     scores = []

#     # QUANTUM-EFFICIENT CROSS-VALIDATION
#     for fold_index, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
#         X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
#         y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

#         # INITIALIZE GPU-ACCELERATED MODEL
#         model = XGBClassifier(
#             **params,
#             early_stopping_rounds=50,  # QUANTUM EFFICIENCY BOOST
#             eval_metric=['mlogloss', 'merror']
#         )

#         # QUANTUM-ACCELERATED TRAINING
#         model.fit(
#             X_fold_train, y_fold_train,
#             eval_set=[(X_fold_val, y_fold_val)],
#             verbose=False
#         )

#         y_pred = model.predict(X_fold_val)
#         score = f1_score(y_fold_val, y_pred, average='macro')
#         scores.append(score)

#         trial.report(np.mean(scores), step=fold_index)
#         if trial.should_prune():
#             raise TrialPruned()

#         quantum_memory_cleanup()  # CRITICAL: PREVENT QUANTUM DECOHERENCE

#     return np.mean(scores)

# # QUANTUM-ENHANCED STUDY CONFIGURATION
# study = optuna.create_study(
#     direction='maximize',
#     sampler=optuna.samplers.TPESampler(
#         multivariate=True,
#         constant_liar=True
#     ),
#     pruner=optuna.pruners.MedianPruner(
#         n_startup_trials=5,
#         n_warmup_steps=5,
#         interval_steps=1
#     )
# )

# # OPTIMIZE WITH QUANTUM PARALLELIZATION
# study.optimize(
#     objective,
#     n_trials=2,
#     n_jobs=2,  # OPTIMAL FOR GPU WORKLOAD
#     timeout=3600 * 8  # 8-HOUR QUANTUM SAFETY NET
# )

# quantum_memory_cleanup()  # FINAL QUANTUM PURIFICATION

# # TRAIN FINAL MODEL WITH QUANTUM ACCELERATION
# final_model = XGBClassifier(
#     **study.best_params,
#     tree_method='gpu_hist',
#     predictor='gpu_predictor',
#     objective='multi:softprob',
#     random_state=42
# )

# final_model.fit(X_train, y_train)

wider ranges are mathematically optimal for Bayesian optimization as long as they respect physical constraints.

Snoek et al. (2012) - "Practical Bayesian Optimization of Machine Learning Algorithms"

Bergstra et al. (2011) - "Algorithms for Hyper-Parameter Optimization"

use of L4 GPU from google colab to up njobs for 512 paralells over 10000 trials supporting the paralell bayesian computation with n_startup_trials=1500 n_warmup_steps=15 for strong pruning

In [17]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import numpy as np
from optuna import TrialPruned
import gc
import torch
import logging
from datetime import datetime
from pathlib import Path

# Initialize label encoder
label_encoder = LabelEncoder()

# Define target variable and training data
# Define target variable and training data
y = merged_df['Claim Injury Type']
X_train = analysis_df.astype('float64')  # Convert features to float
y_train = label_encoder.fit_transform(y).astype('int32')  # Convert labels to int

# Initialize logging
log_path = Path(f"quantum_optimization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[logging.FileHandler(log_path), logging.StreamHandler()])


QUANTUM_PARAMS = {
    'max_depth': (5, 25),
    'n_estimators': (100, 1000),
    'learning_rate': (0.01, 1),
    'min_child_weight': (0.1, 10),
    'subsample': (0.1, 1.0),
    'colsample_bytree': (0.1, 1.0),
    'reg_alpha': (0.01, 10.0),
    'reg_lambda': (0.01, 10.0),
    'gamma': (0, 10),
    'scale_pos_weight': [0.1, 50]
}

def objective(trial):
    params = {
        'max_depth': trial.suggest_int('max_depth', *QUANTUM_PARAMS['max_depth']),
        'learning_rate': trial.suggest_float('learning_rate', *QUANTUM_PARAMS['learning_rate'], log=True),
        'min_child_weight': trial.suggest_float('min_child_weight', *QUANTUM_PARAMS['min_child_weight']),
        'subsample': trial.suggest_float('subsample', *QUANTUM_PARAMS['subsample']),
        'colsample_bytree': trial.suggest_float('colsample_bytree', *QUANTUM_PARAMS['colsample_bytree']),
        'n_estimators': trial.suggest_int('n_estimators', *QUANTUM_PARAMS['n_estimators']),
        'reg_alpha': trial.suggest_float('reg_alpha', *QUANTUM_PARAMS['reg_alpha'], log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', *QUANTUM_PARAMS['reg_lambda'], log=True),
        'gamma': trial.suggest_float('gamma', *QUANTUM_PARAMS['gamma']),
        'tree_method': 'gpu_hist' if torch.cuda.is_available() else 'hist',
        'predictor': 'gpu_predictor' if torch.cuda.is_available() else 'auto',
        'objective': 'multi:softmax',
        'eval_metric': ['merror', 'mlogloss'],
        'random_state': 42
    }

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []

    try:
        for fold_idx, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train)):
            X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

            model = XGBClassifier(early_stopping_rounds=50, **params)
            model.fit(X_fold_train, y_fold_train,
                     eval_set=[(X_fold_val, y_fold_val)],
                     verbose=False)

            y_pred = model.predict(X_fold_val)
            score = f1_score(y_fold_val, y_pred, average='macro')
            scores.append(score)

            if fold_idx == 0 or score > max(scores[:-1], default=0):
                logging.info(f"Trial {trial.number}, Fold {fold_idx + 1}: F1={score:.4f}")

            trial.report(np.mean(scores), step=fold_idx)
            if trial.should_prune():
                raise TrialPruned()

    except Exception as e:
        logging.error(f"Trial {trial.number} error: {str(e)}")
        return float('-inf')

    return np.mean(scores)

def train_model():
    study = optuna.create_study(
        direction='maximize',
        sampler=optuna.samplers.TPESampler(),
        pruner=optuna.pruners.MedianPruner(
            n_startup_trials=12,
            n_warmup_steps=3
        )
    )

    study.optimize(
        objective,
        n_trials=250,
        timeout=360000,
        n_jobs=24
    )

    logging.info(f"Best trial: F1={study.best_value:.4f}")
    logging.info(f"Best parameters: {study.best_params}")

    final_model = XGBClassifier(**study.best_params)
    final_model.fit(X_train, y_train)

    return final_model, study.best_params

if __name__ == "__main__":
    model, best_params = train_model()

[I 2024-12-20 17:24:29,435] A new study created in memory with name: no-name-1241ed64-08b7-4b62-964d-e78d7608c69f
[I 2024-12-20 17:29:01,577] Trial 11 finished with value: 0.4031790289044624 and parameters: {'max_depth': 22, 'learning_rate': 0.18590128711035014, 'min_child_weight': 5.501188750423319, 'subsample': 0.47725704979493355, 'colsample_bytree': 0.5753467999026584, 'n_estimators': 204, 'reg_alpha': 8.758657788160276, 'reg_lambda': 0.508794640912184, 'gamma': 8.409122760104895}. Best is trial 11 with value: 0.4031790289044624.
[I 2024-12-20 17:29:37,555] Trial 12 finished with value: 0.4148975401122755 and parameters: {'max_depth': 25, 'learning_rate': 0.40194145034935697, 'min_child_weight': 8.254621836131975, 'subsample': 0.3676105028895724, 'colsample_bytree': 0.9638606234999761, 'n_estimators': 298, 'reg_alpha': 2.718689488655312, 'reg_lambda': 2.8630521758992473, 'gamma': 8.528429848827495}. Best is trial 12 with value: 0.4148975401122755.
[I 2024-12-20 17:29:57,726] Trial 

KeyboardInterrupt: 

**[I 2024-12-20 20:35:24,532] Trial 209 finished with value: 0.44869824600011743 and parameters: {'max_depth': 11, 'learning_rate': 0.056385509248636015, 'min_child_weight': 9.387401545733596, 'subsample': 0.7567602975176431, 'colsample_bytree': 0.7835525670452888, 'n_estimators': 904, 'reg_alpha': 0.14824963150155737, 'reg_lambda': 0.896780930690089, 'gamma': 1.6379154415407122}. Best is trial 209 with value: 0.44869824600011743.**

In [14]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, classification_report
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import numpy as np
from optuna import TrialPruned
from sklearn.model_selection import train_test_split

In [15]:
def process_and_predict(analysis_df, claim_injury_type, test_scaled):
    """
    Implementation with feature importance and additional performance metrics.
    """
    # Label encoding
    le = LabelEncoder()
    y = le.fit_transform(claim_injury_type)

    # Stratified split
    X_train, X_val, y_train, y_val = train_test_split(
        analysis_df, y,
        test_size=0.3,
        random_state=42,
        stratify=y
    )

    model = XGBClassifier(
        objective='multi:softprob',
        random_state=42,
        learning_rate=0.05046195857265063,
        max_depth=14,
        min_child_weight=4.295663382738008,
        subsample=0.5794673021390964,
        colsample_bytree=0.6752893520492427,
        n_estimators=716,
        reg_alpha=0.2207882375290882,
        reg_lambda=0.28791727579162424,
        gamma=1.5556906330098323,
        tree_method="gpu_hist"
    )

    # Model fitting
    model.fit(X_train, y_train)

    # Predictions
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_val)
    test_pred = model.predict(test_scaled)

    # Probability predictions for ROC AUC
    train_pred_proba = model.predict_proba(X_train)
    val_pred_proba = model.predict_proba(X_val)

    # Feature importance calculation
    feature_importance = pd.DataFrame({
        'feature': analysis_df.columns,
        'importance': model.feature_importances_
    })
    feature_importance = feature_importance.sort_values('importance', ascending=False)

    # Expanded metrics
    metrics = {
        'train_f1': f1_score(y_train, train_pred, average='macro'),
        'val_f1': f1_score(y_val, val_pred, average='macro'),
        'train_acc': accuracy_score(y_train, train_pred),
        'val_acc': accuracy_score(y_val, val_pred),
        'train_precision': precision_score(y_train, train_pred, average='macro'),
        'val_precision': precision_score(y_val, val_pred, average='macro'),
        'train_recall': recall_score(y_train, train_pred, average='macro'),
        'val_recall': recall_score(y_val, val_pred, average='macro'),
        'train_roc_auc': roc_auc_score(y_train, train_pred_proba, multi_class='ovr'),
        'val_roc_auc': roc_auc_score(y_val, val_pred_proba, multi_class='ovr')
    }

    # Classification reports
    print("\nTRAINING SET PERFORMANCE METRICS:")
    print(classification_report(y_train, train_pred))
    print("\nVALIDATION SET PERFORMANCE METRICS:")
    print(classification_report(y_val, val_pred))

    # Feature importance display
    print("\nFEATURE IMPORTANCE RANKING:")
    print(feature_importance.to_string(index=False))

    # Class distribution analysis
    print("\nCLASS DISTRIBUTION ANALYSIS:")
    print("Training Set Class Distribution:")
    print(pd.Series(y_train).value_counts(normalize=True))
    print("\nValidation Set Class Distribution:")
    print(pd.Series(y_val).value_counts(normalize=True))

    # Transform predictions
    test_pred_labels = le.inverse_transform(test_pred)

    return {
        'metrics': metrics,
        'test_predictions': test_pred_labels,
        'model': model,
        'label_encoder': le,
        'feature_importance': feature_importance
    }

# Execute analysis
try:
    results = process_and_predict(analysis_df, claim_injury_type, test_scaled)

    print("\nPERFORMANCE METRICS SUMMARY:")
    print(f"Train Accuracy: {results['metrics']['train_acc']:.4f}")
    print(f"Validation Accuracy: {results['metrics']['val_acc']:.4f}")
    print(f"Train F1-macro: {results['metrics']['train_f1']:.4f}")
    print(f"Validation F1-macro: {results['metrics']['val_f1']:.4f}")
    print(f"Train Precision: {results['metrics']['train_precision']:.4f}")
    print(f"Validation Precision: {results['metrics']['val_precision']:.4f}")
    print(f"Train Recall: {results['metrics']['train_recall']:.4f}")
    print(f"Validation Recall: {results['metrics']['val_recall']:.4f}")
    print(f"Train ROC AUC: {results['metrics']['train_roc_auc']:.4f}")
    print(f"Validation ROC AUC: {results['metrics']['val_roc_auc']:.4f}")

    # Save predictions
    predictions_df = pd.DataFrame({
        'Predicted_Claim_Injury_Type': results['test_predictions']
    })
    predictions_df.to_csv('test_predictions3.csv', index=False)

    # Save feature importance
    results['feature_importance'].to_csv('feature_importance3.csv', index=False)

except Exception as e:
    print(f"Error detected: {str(e)}")
    raise

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



TRAINING SET PERFORMANCE METRICS:
              precision    recall  f1-score   support

           0       0.87      0.68      0.77      8733
           1       0.88      0.99      0.93    203754
           2       0.92      0.29      0.44     48234
           3       0.83      0.94      0.88    103955
           4       0.87      0.80      0.83     33796
           5       1.00      0.28      0.44      2948
           6       1.00      0.19      0.32        68
           7       0.91      0.74      0.82       329

    accuracy                           0.86    401817
   macro avg       0.91      0.61      0.68    401817
weighted avg       0.87      0.86      0.84    401817


VALIDATION SET PERFORMANCE METRICS:


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.75      0.51      0.61      3743
           1       0.85      0.98      0.91     87324
           2       0.51      0.10      0.16     20672
           3       0.73      0.87      0.79     44552
           4       0.70      0.61      0.65     14484
           5       0.23      0.00      0.01      1263
           6       0.00      0.00      0.00        29
           7       0.53      0.33      0.41       141

    accuracy                           0.79    172208
   macro avg       0.54      0.42      0.44    172208
weighted avg       0.76      0.79      0.76    172208


FEATURE IMPORTANCE RANKING:
                         feature  importance
       Attorney/Representative_Y    0.211774
                     Received_C2    0.208763
             Average Weekly Wage    0.113654
                     IME-4 Count    0.063257
                    Hearing_held    0.050833
Alternative Dispute Resolution_Y    0.044845
     

In [ ]:
# def process_and_predict(analysis_df, claim_injury_type, test_scaled):
#     """
#     Implementation with feature importance and additional performance metrics.
#     """
#     # Label encoding
#     le = LabelEncoder()
#     y = le.fit_transform(claim_injury_type)

#     # Stratified split
#     X_train, X_val, y_train, y_val = train_test_split(
#         analysis_df, y,
#         test_size=0.3,
#         random_state=42,
#         stratify=y
#     )

# #     # Model 0.?
# #     model = XGBClassifier(
# #     objective='multi:softprob',  # Multi-class probability output
# #     random_state=42,             # Reproducibility
# #     learning_rate=0.075,          # Slower learning for better convergence
# #     max_depth=18,                # Deep trees for maximum pattern capture
# #     min_child_weight=8,          # Allow smaller splits for deeper exploration
# #     subsample=0.8,               # Higher row sampling for more dataset richness
# #     colsample_bytree=0.6,        # More feature sampling for diversity
# #     n_estimators=2000,           # More trees for finer learning
# #     reg_alpha=1,                 # Stronger L1 regularization
# #     reg_lambda=1.5,                # Stronger L2 regularization for stability
# #     tree_method="gpu-hist",      # Faster GPU acceleration for large datasets
# #     grow_policy="lossguide",     # Favors deeper trees guided by loss reduction
# #     max_leaves=512,              # Wider trees for more splits
# #     booster="gbtree",            # Standard gradient-boosted tree
# #     gamma=0.5                    # Less penalty on unnecessary splits
# # )
# # 55 finished with value: 0.44430262591940056 and parameters: {'max_depth': 14, 'learning_rate': 0.05046195857265063,
# #                                                              'min_child_weight': 4.295663382738008, 'subsample': 0.5794673021390964,
# #                                                              'colsample_bytree': 0.6752893520492427,
# #                                                              'n_estimators': 716, 'reg_alpha': 0.2207882375290882,
# #                                                                                                                          'reg_lambda': 0.28791727579162424, 'gamma': 1.5556906330098323}
#       # Model 0.?

#         model = XGBClassifier(
#             objective='multi:softprob',
#             random_state=42,
#             learning_rate=0.05046195857265063,
#             max_depth=14,
#             min_child_weight=4.295663382738008,
#             subsample=0.5794673021390964,
#             colsample_bytree=0.6752893520492427,
#             n_estimators=716,
#             reg_alpha=0.2207882375290882,
#             reg_lambda=0.28791727579162424,
#             gamma = 1.5556906330098323,
#             tree_method="gpu-hist"       # Faster GPU acceleration for large datasets
#         )

#       # # Model 0.48
#       # model = XGBClassifier(
#       #     objective='multi:softprob',
#       #     random_state=42,
#       #     learning_rate=0.075,
#       #     max_depth=18,      #
#       #     min_child_weight=8,
#       #     subsample=0.8,
#       #     colsample_bytree=0.6,
#       #     n_estimators=2000,
#       #     reg_alpha=1.0,
#       #     reg_lambda=1.5,
#       #     tree_method="gpu-hist      # Faster GPU acceleration for large datasets
#       # )


#     # # Model 0.47
#     # model = XGBClassifier(
#     #     objective='multi:softprob',
#     #     random_state=42,
#     #     learning_rate=0.075,
#     #     max_depth=12,
#     #     min_child_weight=8,
#     #     subsample=0.8,
#     #     colsample_bytree=0.6,
#     #     n_estimators=2000,
#     #     reg_alpha=1.0,
#     #     reg_lambda=1.5
#     # )

#     # # this is 0.451
#     # model = XGBClassifier(
#     #     objective='multi:softprob',
#     #     random_state=42,
#     #     learning_rate=0.03,
#     #     max_depth=7,
#     #     min_child_weight=5,
#     #     subsample=0.8,
#     #     colsample_bytree=0.6,
#     #     n_estimators=1000,
#     #     reg_alpha=1.0,
#     #     reg_lambda=1.5
#     # )


#   # # 0.42
#   #   model = XGBClassifier(
#   #       objective='multi:softprob',
#   #       random_state=42,
#   #       learning_rate=0.02,  # Reduced for better convergence
#   #       max_depth=9,         # Increased to capture complex patterns
#   #       min_child_weight=6,  # Adjusted for stability
#   #       subsample=0.85,      # Slightly higher to use more data in each tree
#   #       colsample_bytree=0.7,  # Increased to utilize the expanded feature space
#   #       n_estimators=1500,   # More rounds to compensate for reduced learning rate
#   #       reg_alpha=0.5,       # Lowered regularization for flexibility
#   #       reg_lambda=1.0       # Reduced to prevent underfitting
#   #   )

#     # Model fitting
#     model.fit(X_train, y_train)

#     # Predictions
#     train_pred = model.predict(X_train)
#     val_pred = model.predict(X_val)
#     test_pred = model.predict(test_scaled)

#     # Probability predictions for ROC AUC
#     train_pred_proba = model.predict_proba(X_train)
#     val_pred_proba = model.predict_proba(X_val)

#     # Feature importance calculation
#     feature_importance = pd.DataFrame({
#         'feature': analysis_df.columns,
#         'importance': model.feature_importances_
#     })
#     feature_importance = feature_importance.sort_values('importance', ascending=False)

#     # Expanded metrics
#     metrics = {
#         'train_f1': f1_score(y_train, train_pred, average='macro'),
#         'val_f1': f1_score(y_val, val_pred, average='macro'),
#         'train_acc': accuracy_score(y_train, train_pred),
#         'val_acc': accuracy_score(y_val, val_pred),
#         'train_precision': precision_score(y_train, train_pred, average='macro'),
#         'val_precision': precision_score(y_val, val_pred, average='macro'),
#         'train_recall': recall_score(y_train, train_pred, average='macro'),
#         'val_recall': recall_score(y_val, val_pred, average='macro'),
#         'train_roc_auc': roc_auc_score(y_train, train_pred_proba, multi_class='ovr'),
#         'val_roc_auc': roc_auc_score(y_val, val_pred_proba, multi_class='ovr')
#     }

#     # Classification reports
#     print("\nTRAINING SET PERFORMANCE METRICS:")
#     print(classification_report(y_train, train_pred))
#     print("\nVALIDATION SET PERFORMANCE METRICS:")
#     print(classification_report(y_val, val_pred))

#     # Feature importance display
#     print("\nFEATURE IMPORTANCE RANKING:")
#     print(feature_importance.to_string(index=False))

#     # Class distribution analysis
#     print("\nCLASS DISTRIBUTION ANALYSIS:")
#     print("Training Set Class Distribution:")
#     print(pd.Series(y_train).value_counts(normalize=True))
#     print("\nValidation Set Class Distribution:")
#     print(pd.Series(y_val).value_counts(normalize=True))

#     # Transform predictions
#     test_pred_labels = le.inverse_transform(test_pred)

#     return {
#         'metrics': metrics,
#         'test_predictions': test_pred_labels,
#         'model': model,
#         'label_encoder': le,
#         'feature_importance': feature_importance
#     }

# # Execute analysis
# try:
#     results = process_and_predict(analysis_df, claim_injury_type, test_scaled)

#     print("\nPERFORMANCE METRICS SUMMARY:")
#     print(f"Train Accuracy: {results['metrics']['train_acc']:.4f}")
#     print(f"Validation Accuracy: {results['metrics']['val_acc']:.4f}")
#     print(f"Train F1-macro: {results['metrics']['train_f1']:.4f}")
#     print(f"Validation F1-macro: {results['metrics']['val_f1']:.4f}")
#     print(f"Train Precision: {results['metrics']['train_precision']:.4f}")
#     print(f"Validation Precision: {results['metrics']['val_precision']:.4f}")
#     print(f"Train Recall: {results['metrics']['train_recall']:.4f}")
#     print(f"Validation Recall: {results['metrics']['val_recall']:.4f}")
#     print(f"Train ROC AUC: {results['metrics']['train_roc_auc']:.4f}")
#     print(f"Validation ROC AUC: {results['metrics']['val_roc_auc']:.4f}")

#     # Save predictions
#     predictions_df = pd.DataFrame({
#         'Predicted_Claim_Injury_Type': results['test_predictions']
#     })
#     predictions_df.to_csv('test_predictions3.csv', index=False)

#     # Save feature importance
#     results['feature_importance'].to_csv('feature_importance3.csv', index=False)

# except Exception as e:
#     print(f"Error detected: {str(e)}")
#     raise

In [16]:
import os
import json
import pandas as pd
import subprocess

# Create Kaggle credentials
credentials = {"username":"chielgroeneveld","key":"a823db81aa0c4aa800e0d8ad825fa855"}

# Ensure directory exists and write credentials
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump(credentials, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Read the files and create submission
sample_submission = pd.read_csv('sample_submission.csv')
test_predictions = pd.read_csv('test_predictions3.csv')

# Copy predictions into sample submission format
sample_submission['Claim Injury Type'] = test_predictions['Predicted_Claim_Injury_Type']

# Save as submission file
sample_submission.to_csv('submission.csv', index=False)

# Submit to Kaggle
try:
    result = subprocess.run(
        'kaggle competitions submit -c to-grant-or-not-to-grant -f submission.csv -m "Group19__Version08.csv"',
        shell=True,
        capture_output=True,
        text=True,
        check=True
    )
    print("Submission Output:", result.stdout)
    print("Submission Successful!")
except subprocess.CalledProcessError as e:
    print("Error:", e.stderr)
    print("Return Code:", e.returncode)

FileNotFoundError: [Errno 2] No such file or directory: 'sample_submission.csv'